In [11]:
from operator import itemgetter
from datetime import timedelta
import datetime

from itertools import groupby
from icalendar import Calendar

import ipywidgets as widgets

parcours_dict = {"Gr302": "AII",
"Gr303": "AII",
"Gr304": "AII",
"Gr305": "EME",
"Gr306": "EME",
"Gr307": "ESE",
"Gr308": "ESE"
}

lectures = ['AG', 'PApp1', 'PAPP3', 'OML3', 'R11', 'AM10', 'AM11', 'IU-1', 'IU1', 'SCOT']
parcours = ['AII', 'EME', 'ESE']

def validate_date(date_text):
    try:
        datetime.datetime.strptime(date_text, '%Y-%m-%d')
        return True
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")
        return False

def calculate_time(event):
    start = event['DTSTART'].dt
    end = event['DTEND'].dt
    return end - start

def lecturize(event, lectures):
    summary = str(event['SUMMARY'])
    return [lecture for lecture in lectures if lecture in summary]

def time_per_lecture_old(events):
    lecture_name = itemgetter(0)
    sorted_events = sorted(events, key=lecture_name)

    for key, group in groupby(sorted_events, lecture_name):
        #print(sum(map(itemgetter(1), group), timedelta()))
        yield (tuple(key), sum(map(itemgetter(1), group), timedelta()))

def parcourize(event, parcours_dict):
    """associate a parcours (AII, EME or ESE) to each lecture"""
    DESCRIPTION = str(event['DESCRIPTION'])
    group = DESCRIPTION.split("\n")[2]
    test = group in parcours_dict.keys()

    if group in parcours_dict.keys():
        track = parcours_dict[group]
    else:
        track = "BUT1"
    return track

def time_per_lecture(events):
    sorted_events = sorted(events, key=itemgetter(0, 2))
    for (lecture_name, parcours), batch in groupby(sorted_events, key=itemgetter(0, 2)):
        yield (parcours, lecture_name, sum(map(itemgetter(1), batch), timedelta()) )

        
def naive_event_datetime(ev, date_format_str):
    event_datetime = datetime.datetime.strptime(ev['DTSTART'].dt.strftime(date_format_str), date_format_str)
    return event_datetime


uploader = widgets.FileUpload()
display(uploader)

out = widgets.Output(layout={'border': '1px solid black'})
out.append_stdout("Cliquer sur le bouton et sélectionner votre fichier .ics extrait à partir d'ADE."+"\n")

@out.capture(clear_output=True)
def run(change):
    
    # Create a calendar object using the uploaded file
    for k in uploader.value:
        out.append_stdout(k)
        for el in k:
            print(el)
            print("ok")
        
    k = list(uploader.value.keys())[-1]
    file_read = uploader.value[k]["content"]
    #print(file_read)
    out.append_stdout(file_read)
    cal = Calendar.from_ical(file_read)

    # create the list of tuples (lecture_event, time) from the calendar
    sessions = [(lecturize(e, lectures), calculate_time(e), parcourize(e, parcours_dict)) for e in cal.walk('vevent')]
    
    # Create a generator to group the amount of time per lecture type
    used_time = time_per_lecture(sessions)

    # Sum the entire amount of time including all lectures
    total_time = 0
    for el in used_time:
        track = el[0]
        lecture = el[1]
        duration = el[2]
        total_time += duration.total_seconds() / 3600
        out.append_stdout('parcours {} module {} : \t{}h'.format(track, lecture, duration.total_seconds() / 3600)+"\n")

    out.append_stdout('============='+"\n")
    out.append_stdout('TOTAL\t{}h'.format(total_time)+"\n")
    
    
uploader.observe(run, names="value")



FileUpload(value={}, description='Upload')

In [12]:
out

Output(layout=Layout(border='1px solid black'), outputs=({'output_type': 'stream', 'name': 'stdout', 'text': "…